# Interfaces con otros lenguajes: C++ 

Existen varias formas de utilizar bibliotecas o códigos hechos en C desde Python. Nosotros veremos el uso de `Ctypes`, sin embargo existen otras alternativas como  [Cython](https://cython.org/), [CFFI](https://cffi.readthedocs.io/en/latest/), [pybind11](https://pybind11.readthedocs.io/en/stable/) y [Boost.Python](https://www.boost.org/doc/libs/1_70_0/libs/python/doc/html/index.html). 


## Ejemplo 1: Problema a resolver

Supongamos que queremos resolver el problema de la rotación de vectores en el espacio usando los tres ángulos de Euler.



In [1]:
import numpy as np

In [2]:
pwd

'/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/cursos/curso-python'

Si ya tenemos un módulo donde están programadas las funciones necesarias 

es fácil utilizarlas. Las importamos y utilizamos

## Interfaces con C

Veamos cómo trabajar si tenemos el código para realizar las rotaciones en C.

### Primer ejemplo: Nuestro código

El código en C que tenemos es:

```c
 typedef struct {
      float m[3][3];
    } m3x3;

    typedef struct {
      float a[3];
    } v3;
    
...

float * rotate(float angles[3], float *v, int N){

        m3x3 R = matrix_rotation(angles);
        
        float* y = (float*)malloc(3*N*sizeof(float));
        v3 p;

        printf("%p\n",y);
        for(int i=0; i<N; i++){
            // p = &y[i*3];
            p = matmul3(R,&v[i*3]);
            y[i*3+0] = p.a[0];
            y[i*3+1] = p.a[1];
            y[i*3+2] = p.a[2];
            // printf("%6.3f %6.3f %6.3f \n",y[i*3+0],y[i*3+1],y[i*3+2]);
        }
        return y;


  }

```

In [3]:
cd interfacing_Cpp

/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/cursos/curso-python/interfacing_Cpp


In [4]:
!cat test.cpp

class Test{
     private:
        int n;
     public:
        Test(int k){
            n=k;
        }
        void setInt(int k){
            n = k;
        }
        int getInt(){
            return n;
        }
};

extern "C" 
{
    // include below each method you want to make visible outside
    Test* init(int k) {return new Test(k);}
    void setInt(Test *self, int k) {self->setInt(k);}
    int getInt(Test *self) {return self->getInt();}
    
    // Add the declaration '__declspec(dllexport)' before each function in Windows
}

### CTypes

No vamos a usar directamente `Ctypes`, sino a través de `NumPy`, que provee algunas funciones convenientes para acceder al código C.

El primer paso es compilar nuestro código y generar una biblioteca:
```bash
$ gcc -fpic -Wall -shared rotacion.c -o librotacion.so
```

Si uno trabaja en Windows, generará una dll

```cmd
cl.exe -c rotacion.c
link.exe /DLL /OUT:rotacion.dll
```


In [5]:
# !gcc -fpic -Wall -shared rotacion.c -o librotacion.so
!g++ -shared test.cpp -o libtest.so



In [6]:
!ls

libtest.dll libtest.so  test.cpp


En segundo lugar, importamos el módulo `ctypeslib`

In [7]:
import numpy.ctypeslib as ctl
import ctypes

Este módulo nos provee de la función `load_library` para importar la biblioteca

In [ ]:
lib = ctypes.CDLL('./libtest.dll', **mode)


class Test(object):
    def __init__(self, val: int):
        # Declare input and output types for each method you intend to use
        lib.init.argtypes = [ctypes.c_int]
        lib.init.restype = ctypes.c_void_p

        lib.setInt.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.setInt.restype = ctypes.c_void_p

        lib.getInt.argtypes = [ctypes.c_void_p]
        lib.getInt.restype = ctypes.c_int

        self.obj = lib.init(val)

    def setInt(self, n):
        lib.setInt(self.obj, n)
    
    def getInt(self):
        return lib.getInt(self.obj)


In [8]:
lib = ctl.load_library('libtest.so','.')

In [9]:
class Test(object):
    def __init__(self, val: int):
        # Declare input and output types for each method you intend to use
        lib.init.argtypes = [ctl.c_intp]
        lib.init.restype = ctypes.c_void_p

        lib.setInt.argtypes = [ctypes.c_void_p, ctl.c_intp]
        lib.setInt.restype = ctypes.c_void_p

        lib.getInt.argtypes = [ctypes.c_void_p]
        lib.getInt.restype = ctl.c_intp

        self.obj = lib.init(val)

    def setInt(self, n):
        lib.setInt(self.obj, n)
    
    def getInt(self):
        return lib.getInt(self.obj)

In [10]:
T1 = Test(12)
print(T1.getInt())
T1.setInt(32)
print(T1.getInt())

12
32
